In [1]:
import numpy as np
import pandas as pd
import re
from datetime import datetime, date, time
import locale
import os

locale.setlocale(locale.LC_TIME, 'ru_RU.UTF-8')

'ru_RU.UTF-8'

In [2]:
# source = pd.read_excel('2020.xlsx', sheet_name='(2)', header=1)
# source

In [3]:
def get_values(global_start, start_ind, source, sheet_name):
    """
    Читает данные из Excel и возвращает DataFrame
    
    global_start: начальный индекс где содержится имя участка - как правило = 0
    start_ind: индекс начала блока данных
    source: DataFrame откуда будут данные читаться
    sheet_name: имя текущего листа с которого идет обработка данных
    """
    # индексы строк для месяцев в каждои блоке данных
    months = [1, 2, 3, 5, 6, 7, 9, 10, 11, 13, 14, 15]

    # Разбираем год из строки
    year = re.search(r'\d{4}', source['Unnamed: 1'][1]).group(0)

    # Формируем имя участка
    name1 = source['Unnamed: 0'][global_start] + ': ' + source['Unnamed: 0'][start_ind]
    
    res = []
    for month_num in months:
        
        # Название участка
        line_name = name1
        
        # метка времени в формате datetime
        date_str = year + '-' + str(np.char.title(source['Unnamed: 0'][start_ind+month_num]))[:3] + '-1'
        date_time = datetime.strptime(date_str,'%Y-%b-%d').date()

        # Факт среднесуточного твпс по месяцу
        # преобразуем в число, если нет - возвращаем None
        fact_day_month_str = source['Unnamed: 1'][start_ind+month_num]
        fact_day_month = float(fact_day_month_str) if (fact_day_month_str != '-') else None
        
        # ТВПС по месяцу без рем работ
        # преобразуем в число, если нет - возвращаем None
        tvps_month_without_rr_str = source['Unnamed: 2'][start_ind+month_num]
        tvps_month_without_rr = float(tvps_month_without_rr_str) if (tvps_month_without_rr_str != '-') else None
        
        # ТВПС по месяцу с учетом рем работ
        # преобразуем в число, если нет - возвращаем None
        tvps_month_with_rr_str = source['Unnamed: 3'][start_ind+month_num]
        tvps_month_with_rr = float(tvps_month_with_rr_str) if (tvps_month_with_rr_str != '-') else None
        
        
        # ТВП по месяцу без рем работ
        # преобразуем в число, если нет - возвращаем None
        tvp_month_without_rr_str = source['Unnamed: 5'][start_ind+month_num]
        tvp_month_without_rr = float(tvp_month_without_rr_str) if (tvp_month_without_rr_str != '-') else None
        
        # ТВП по месяцу с учетом рем работ
        # преобразуем в число, если нет - возвращаем None
        tvp_month_with_rr_str = source['Unnamed: 6'][start_ind+month_num]
        tvp_month_with_rr = float(tvp_month_with_rr_str) if (tvp_month_with_rr_str != '-') else None
        
        # Факт по месяцу суммарно
        # преобразуем в число, если нет - возвращаем None
        fact_month_sum_str = source['Unnamed: 7'][start_ind+month_num]
        fact_month_sum = float(fact_month_sum_str) if (fact_month_sum_str != '-') else None
        
        
        line_id = re.search(r'(\d+)', sheet_name).group(0)
        
        
        
        res.append([sheet_name,line_id, line_name, date_time, fact_day_month, tvps_month_with_rr,
                         tvps_month_without_rr, tvp_month_with_rr, tvp_month_without_rr, fact_month_sum])
        
    return res

In [4]:
def add_data_in_df(global_start, start_ind, data, source, sheet_name):
    """
    Дергает get_values и складынвает полученный результат в общий DataFrame
    
    global_start: начальный индекс где содержится имя участка - как правило = 0
    start_ind: индекс начала блока данных
    data: DataFrame куда будут записываться данные
    source: DataFrame откуда будут данные читаться
    sheet_name: имя текущего листа с которого идет обработка данных
    """
    for sl in get_values(global_start, start_ind, source, sheet_name):
        data.loc[len(data)] = sl

In [5]:
def read_exel_passports(file_name, data):
    """
    Читает данные из Excel паспорта участка, определяет индексы блоков данных
    и вызывает add_data_in_df
    
    file_name: Имя файла Excel c паспортами за определнный год
    data: DataFrame куда нужно добавить данные
    """
    # Получаем список листов
    try:
        xl = pd.ExcelFile(file_name)
        sheet_names = xl.sheet_names
    except:
        print(f'!!! Exception: {file_name}')
    

    # Проходим по всем листам в книге и добавляем данные в датафрейм
    for sheet_name in sheet_names[2:]:
        source = pd.read_excel(file_name, sheet_name=sheet_name, header=1)

        # Определяем блоки данных для участка
        line_indexes = source.loc[(source['Unnamed: 1'].isna() & source['Unnamed: 2'].isna() & 
                                   source['Unnamed: 3'].isna() & source['Unnamed: 4'].isna() & 
                                   source['Unnamed: 5'].isna() & source['Unnamed: 6'].isna())]['Unnamed: 0'].index[1:]

        # Проходим по всем блокам данных для участка и добавляем их в DataFrame
        for start_index in line_indexes:
            add_data_in_df(0, start_index, data, source, sheet_name)
#             print(f'File: {file_name}, sheet: {sheet_name}, line: {source["Unnamed: 0"][0] + ": " + source["Unnamed: 0"][start_index]}')
        print(f'File: {file_name}, sheet: {sheet_name}')

In [7]:

# создаем DataFrame и определяем необходимые столбцы
d = pd.DataFrame()
d = pd.DataFrame(columns=['sheet_name','line_id', 'line_name', 'time', 'fact_day_month', 'tvps_with_rr', 'tvps_without_rr', 'tvp_with_rr', 'tvp_without_rr', 'fact_month_sum'])

# Получаем список xlsx файлов в текущем каталоге для дальнейшего разбора данных
files = []
for file_name in os.listdir(path="."):
    if (file_name.endswith(".xls") or file_name.endswith(".xlsx")):
        read_exel_passports(file_name, d)

print(f'Read {d.shape[0]} lines')
os.system('say "Calculation complete"')

File: ТВПС 2010.xlsx, sheet: (2)
File: ТВПС 2010.xlsx, sheet: (3)
File: ТВПС 2010.xlsx, sheet: (4)
File: ТВПС 2010.xlsx, sheet: (5)
File: ТВПС 2010.xlsx, sheet: (6)
File: ТВПС 2010.xlsx, sheet: (7)
File: ТВПС 2010.xlsx, sheet: (8)
File: ТВПС 2010.xlsx, sheet: (9)
File: ТВПС 2010.xlsx, sheet: (10)
File: ТВПС 2010.xlsx, sheet: (11)
File: ТВПС 2010.xlsx, sheet: (13)
File: ТВПС 2010.xlsx, sheet: (14)
File: ТВПС 2010.xlsx, sheet: (15)
File: ТВПС 2010.xlsx, sheet: (86)
File: ТВПС 2010.xlsx, sheet: (84)
File: ТВПС 2010.xlsx, sheet: (16)
File: ТВПС 2010.xlsx, sheet: (17)
File: ТВПС 2010.xlsx, sheet: (18)
File: ТВПС 2010.xlsx, sheet: (19)
File: ТВПС 2010.xlsx, sheet: (20)
File: ТВПС 2010.xlsx, sheet: (21)
File: ТВПС 2010.xlsx, sheet: (22)
File: ТВПС 2010.xlsx, sheet: (23)
File: ТВПС 2010.xlsx, sheet: (24)
File: ТВПС 2010.xlsx, sheet: (25)
File: ТВПС 2010.xlsx, sheet: (26)
File: ТВПС 2010.xlsx, sheet: (27)
File: ТВПС 2010.xlsx, sheet: (28)
File: ТВПС 2010.xlsx, sheet: (29)
File: ТВПС 2010.xlsx, 

File: ТВПС 2011.xlsx, sheet: (37)
File: ТВПС 2011.xlsx, sheet: (87)
File: ТВПС 2011.xlsx, sheet: (88)
File: ТВПС 2011.xlsx, sheet: (38)
File: ТВПС 2011.xlsx, sheet: (39)
File: ТВПС 2011.xlsx, sheet: (40)
File: ТВПС 2011.xlsx, sheet: (41)
File: ТВПС 2011.xlsx, sheet: (42)
File: ТВПС 2011.xlsx, sheet: (43)
File: ТВПС 2011.xlsx, sheet: (44)
File: ТВПС 2011.xlsx, sheet: (45)
File: ТВПС 2011.xlsx, sheet: (46)
File: ТВПС 2011.xlsx, sheet: (47)
File: ТВПС 2011.xlsx, sheet: (48)
File: ТВПС 2011.xlsx, sheet: (49)
File: ТВПС 2011.xlsx, sheet: (50)
File: ТВПС 2011.xlsx, sheet: (51)
File: ТВПС 2011.xlsx, sheet: (52)
File: ТВПС 2011.xlsx, sheet: (53)
File: ТВПС 2011.xlsx, sheet: (54)
File: ТВПС 2011.xlsx, sheet: (55)
File: ТВПС 2011.xlsx, sheet: (56)
File: ТВПС 2011.xlsx, sheet: (57)
File: ТВПС 2011.xlsx, sheet: (58)
File: ТВПС 2011.xlsx, sheet: (59)
File: ТВПС 2011.xlsx, sheet: (60)
File: ТВПС 2011.xlsx, sheet: (61)
File: ТВПС 2011.xlsx, sheet: (62)
File: ТВПС 2011.xlsx, sheet: (64)
File: ТВПС 201

File: 2020.xlsx, sheet: (83)
File: 2020.xlsx, sheet: (85)
File: 2020.xlsx, sheet: (90)
File: 2020.xlsx, sheet: (91)
File: 2020.xlsx, sheet: (92)
File: 2020.xlsx, sheet: (93)
File: 2020.xlsx, sheet: (94)
File: 2020.xlsx, sheet: (95)
File: 2020.xlsx, sheet: (96)
File: 2020.xlsx, sheet: (97)
File: 2020.xlsx, sheet: (98)
File: 2020.xlsx, sheet: (99)
File: 2020.xlsx, sheet: (100)
File: 2020.xlsx, sheet: (101)
File: 2020.xlsx, sheet: (102)
File: 2020.xlsx, sheet: (103)
File: 2020.xlsx, sheet: (104)
File: 2020.xlsx, sheet: (105)
File: 2020.xlsx, sheet: (106)
File: 2020.xlsx, sheet: (107)
File: 2020.xlsx, sheet: (108)
File: 2020.xlsx, sheet: (1)
File: ТВПС 2016.xlsx, sheet: (2)
File: ТВПС 2016.xlsx, sheet: (3)
File: ТВПС 2016.xlsx, sheet: (4)
File: ТВПС 2016.xlsx, sheet: (5)
File: ТВПС 2016.xlsx, sheet: (6)
File: ТВПС 2016.xlsx, sheet: (7)
File: ТВПС 2016.xlsx, sheet: (8)
File: ТВПС 2016.xlsx, sheet: (9)
File: ТВПС 2016.xlsx, sheet: (10)
File: ТВПС 2016.xlsx, sheet: (11)
File: ТВПС 2016.xlsx, 

File: ТВПС 2014.xlsx, sheet: (86)
File: ТВПС 2014.xlsx, sheet: (84)
File: ТВПС 2014.xlsx, sheet: (16)
File: ТВПС 2014.xlsx, sheet: (17)
File: ТВПС 2014.xlsx, sheet: (18)
File: ТВПС 2014.xlsx, sheet: (19)
File: ТВПС 2014.xlsx, sheet: (20)
File: ТВПС 2014.xlsx, sheet: (21)
File: ТВПС 2014.xlsx, sheet: (22)
File: ТВПС 2014.xlsx, sheet: (23)
File: ТВПС 2014.xlsx, sheet: (24)
File: ТВПС 2014.xlsx, sheet: (25)
File: ТВПС 2014.xlsx, sheet: (26)
File: ТВПС 2014.xlsx, sheet: (27)
File: ТВПС 2014.xlsx, sheet: (28)
File: ТВПС 2014.xlsx, sheet: (29)
File: ТВПС 2014.xlsx, sheet: (30)
File: ТВПС 2014.xlsx, sheet: (31)
File: ТВПС 2014.xlsx, sheet: (32)
File: ТВПС 2014.xlsx, sheet: (33)
File: ТВПС 2014.xlsx, sheet: (34)
File: ТВПС 2014.xlsx, sheet: (35)
File: ТВПС 2014.xlsx, sheet: (36)
File: ТВПС 2014.xlsx, sheet: (37)
File: ТВПС 2014.xlsx, sheet: (87)
File: ТВПС 2014.xlsx, sheet: (88)
File: ТВПС 2014.xlsx, sheet: (38)
File: ТВПС 2014.xlsx, sheet: (39)
File: ТВПС 2014.xlsx, sheet: (40)
File: ТВПС 201

File: ТВПС 2012.xlsx, sheet: (43)
File: ТВПС 2012.xlsx, sheet: (44)
File: ТВПС 2012.xlsx, sheet: (45)
File: ТВПС 2012.xlsx, sheet: (46)
File: ТВПС 2012.xlsx, sheet: (47)
File: ТВПС 2012.xlsx, sheet: (48)
File: ТВПС 2012.xlsx, sheet: (49)
File: ТВПС 2012.xlsx, sheet: (50)
File: ТВПС 2012.xlsx, sheet: (51)
File: ТВПС 2012.xlsx, sheet: (52)
File: ТВПС 2012.xlsx, sheet: (53)
File: ТВПС 2012.xlsx, sheet: (54)
File: ТВПС 2012.xlsx, sheet: (55)
File: ТВПС 2012.xlsx, sheet: (56)
File: ТВПС 2012.xlsx, sheet: (57)
File: ТВПС 2012.xlsx, sheet: (58)
File: ТВПС 2012.xlsx, sheet: (59)
File: ТВПС 2012.xlsx, sheet: (60)
File: ТВПС 2012.xlsx, sheet: (61)
File: ТВПС 2012.xlsx, sheet: (62)
File: ТВПС 2012.xlsx, sheet: (64)
File: ТВПС 2012.xlsx, sheet: (65)
File: ТВПС 2012.xlsx, sheet: (66)
File: ТВПС 2012.xlsx, sheet: (67)
File: ТВПС 2012.xlsx, sheet: (68)
File: ТВПС 2012.xlsx, sheet: (69)
File: ТВПС 2012.xlsx, sheet: (70)
File: ТВПС 2012.xlsx, sheet: (71)
File: ТВПС 2012.xlsx, sheet: (72)
File: ТВПС 201

0

In [6]:
for file_name in os.listdir(path="."):
    if (file_name.endswith(".xls") or file_name.endswith(".xlsx")):
        print(file_name)

ТВПС 2010.xlsx
ТВПС 2019.xls
ТВПС 2011.xlsx
ТВПС 2018.xls
2020.xlsx
ТВПС 2016.xlsx
ТВПС 2017.xlsx
ТВПС 2014.xlsx
ТВПС 2015.xlsx
ТВПС 2012.xlsx
ТВПС 2013.xlsx


In [10]:
d.to_csv('Passports_data.csv', sep='\t', encoding='utf-8')

In [9]:
d.shape

(46404, 10)